In [ ]:
import pennylane as qml
from pennylane import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import random
import os
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import random
from PIL import Image

In [2]:
def get_category_dict(dataset_path):
    categories = sorted([
        folder for folder in os.listdir(dataset_path)
        if os.path.isdir(os.path.join(dataset_path, folder))
    ])
    return {category: idx for idx, category in enumerate(categories)}

In [3]:
dataset = "Food_Classification"
categories = get_category_dict(dataset)
num_images = 500
image_size = (256, 256) 
print(categories)

{'burger': 0, 'butter_naan': 1, 'chai': 2, 'chapati': 3, 'chole_bhature': 4, 'dal_makhani': 5, 'dhokla': 6, 'fried_rice': 7, 'idli': 8, 'jalebi': 9, 'kaathi_rolls': 10, 'kadai_paneer': 11, 'kulfi': 12, 'masala_dosa': 13, 'momos': 14, 'paani_puri': 15, 'pakode': 16, 'pav_bhaji': 17, 'pizza': 18, 'samosa': 19}


In [4]:
ksize = 31
sigma = 4.0
lambd = 10.0
gamma = 0.5
psi = 0
orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]

In [5]:
def get_image_paths(dataset_dir):
    all_image_paths = []
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_path):
            for file in os.listdir(class_path):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    full_path = os.path.join(class_path, file)
                    all_image_paths.append(full_path)
    return all_image_paths



In [6]:
image_paths = get_image_paths(dataset)[:num_images]

In [7]:
gabor_features = []

In [8]:
for path in tqdm(image_paths, desc="Applying Gabor Filters"):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = cv2.resize(img, image_size)
    filtered_stack = []

    for theta in orientations:
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(img, cv2.CV_32F, kernel)
        filtered_stack.append(filtered)

    gabor_feature = np.stack(filtered_stack, axis=-1)
    gabor_features.append(gabor_feature)

gabor_features = np.array(gabor_features) 

Applying Gabor Filters: 100%|██████████| 500/500 [00:11<00:00, 42.83it/s]


In [9]:
print(gabor_features)

[[[[ 9.68228516e+02  1.20360864e+03  1.58411841e+03  1.20360876e+03]
   [ 9.71933716e+02  1.20652759e+03  1.58493970e+03  1.20652783e+03]
   [ 9.82145691e+02  1.21386267e+03  1.58439343e+03  1.21386292e+03]
   ...
   [ 1.11400928e+03  1.41211523e+03  8.81931030e+02  1.41211499e+03]
   [ 1.11754041e+03  1.41369702e+03  8.85410156e+02  1.41369678e+03]
   [ 1.11949316e+03  1.41415771e+03  8.85642761e+02  1.41415771e+03]]

  [[ 9.67695740e+02  1.20599878e+03  1.52900708e+03  1.20599890e+03]
   [ 9.71268555e+02  1.21512036e+03  1.52989722e+03  1.20207776e+03]
   [ 9.81196106e+02  1.22692505e+03  1.52991296e+03  1.20324902e+03]
   ...
   [ 1.11263257e+03  1.42071570e+03  9.09709595e+02  1.39163916e+03]
   [ 1.11611621e+03  1.41429675e+03  9.13081665e+02  1.40011694e+03]
   [ 1.11809802e+03  1.40751074e+03  9.13502441e+02  1.40751050e+03]]

  [[ 9.64539062e+02  1.21217383e+03  1.38876196e+03  1.21217395e+03]
   [ 9.68272095e+02  1.22566699e+03  1.38978040e+03  1.20210669e+03]
   [ 9.78502319e

In [10]:
def nearest_power_of_2(n):
    return 2 ** int(np.ceil(np.log2(n)))

In [43]:
def reduce_gabor_features(gabor_features, out_dim=4):
    reduced_features = []
    for feat in gabor_features:
        pooled = np.mean(feat, axis=(0, 1)) 
        normalized = np.interp(pooled, (pooled.min(), pooled.max()), (0, np.pi))
        reduced_features.append(normalized[:out_dim])
    return np.array(reduced_features)


In [51]:
X_quantum = reduce_gabor_features(gabor_features, out_dim=9)
print(X_quantum)
print("")
print(X_quantum.shape)
num_qubits = X_quantum.shape[1]

[[0.00000000e+00 3.14016955e+00 4.51633920e-03 3.14159265e+00]
 [0.00000000e+00 3.11992531e+00 2.76419540e-02 3.14159265e+00]
 [0.00000000e+00 3.14138252e+00 1.74587794e-03 3.14159265e+00]
 ...
 [4.62607377e-02 3.13684748e+00 0.00000000e+00 3.14159265e+00]
 [1.72506379e-04 3.14148666e+00 0.00000000e+00 3.14159265e+00]
 [0.00000000e+00 3.09808453e+00 9.93685920e-03 3.14159265e+00]]

(500, 4)


In [56]:
X_flat = X_quantum.flatten() 
print(X_flat)

[0.         3.14016955 0.00451634 ... 3.09808453 0.00993686 3.14159265]


In [57]:
def pad_to_power_of_two(arr):
    length = len(arr)
    target_length = 2**int(np.ceil(np.log2(length)))
    padded = np.zeros(target_length)
    padded[:length] = arr
    return padded

X_padded = pad_to_power_of_two(X_flat)

In [58]:
X_norm = X_padded / np.linalg.norm(X_padded)

In [59]:
num_qubits = int(np.log2(len(X_norm)))
dev = qml.device("default.qubit", wires=num_qubits)

In [65]:
@qml.qnode(dev)
def amplitude_encoding_circuit():
    qml.AmplitudeEmbedding(X_norm, wires=range(num_qubits), normalize=False)
    return qml.state()

In [66]:
output = amplitude_encoding_circuit()
print(output)

[0.00000000e+00+0.j 3.17433163e-02+0.j 4.56547271e-05+0.j ...
 0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
